In [1]:
# import libraries
from tableau_api_lib import TableauServerConnection
from tableau_api_lib.utils import querying, flatten_dict_column
from urllib import parse
import pandas as pd
import io
import os
from datetime import date  

import xlsxwriter
from xlsxwriter.utility import xl_rowcol_to_cell
import datetime

In [ ]:
# define server configuration
config = {
    'tableau_prod' : {
        'server': '', #https://xxx.online.tableau.com
        'api_version': '3.19', 
        #'username': '',
        #'password':,
        'personal_access_token_name': 'xxx', # name of token in Tableau site
        'personal_access_token_secret': 'xxx', # from Tableau site user settings
        'site_name': 'xxx', # site name and url sometimes the same
        'site_url': 'xxx'
    }
}

# create connection
conn = TableauServerConnection(config_json=config, env='tableau_prod')

#response = conn.sign_in() #response = 200 is good
conn.sign_in()

### Datasource IDs

In [ ]:
##This section will give us datasource name and id infor 
# data source info
datasource_df = querying.get_datasources_dataframe(conn)
#print(datasource_df.head(25))
#print(datasource_df.iloc[0])
#print(datasource_df.name.unique())
#datasource_df.dtypes

#create data frame just with name and id
ds_name_id = datasource_df[['name', 'id']]
print(ds_name_id.sort_values(by=['name']))

### Workbooks and Sheets

In [ ]:
### Query server

# list of all the views on server
views_df = querying.get_views_dataframe(conn)
views_df = flatten_dict_column(views_df, keys=["name","id"], col_name="workbook")

# workbook we are interested in
comm_sales = views_df[views_df["workbook_name"] == "xxx"] # filter to the workbook of interest

dashboard_view_id = comm_sales[comm_sales['name'] == 'xxx']['id'].values[0] # actual dashboard
filter_view_id = comm_sales[comm_sales['name'] == 'xxx']['id'].values[0] # sheet with list of values for filtering
report_view_id = comm_sales[comm_sales['name'] == 'xxx']['id'].values[0] # raw data table for csv

# data from sheet in Tableau workbook with the names of filter values
filter_data = conn.query_view_data(view_id=filter_view_id)
filter_df = pd.read_csv(io.StringIO(filter_data.content.decode('utf-8')))
filter_list = list(filter_df['xxx']) # name of field used for filter

In [ ]:
### Outside of loop - Params are very customizable

# specify pdf params
pdf_params = {
    "pdf_orientation": "orientation=Portrait",
    "pdf_layout": "type=A4",
    "filter": None
}
# specify csv params
csv_params = {
    "filter": None
}

#folder paths
path = 'xxx' # wherever you want to land the files


### Example for formatted report (not required)
# date range for title
today = datetime.date.today()
last_sunday_offset = today.weekday() + 1  # convert day format mon-sun=0-6 => sun-sat=0-6 
last_sunday = today - datetime.timedelta(days=last_sunday_offset)
# start and end
date_range_start = last_sunday - datetime.timedelta(days=7)
date_range_end = last_sunday - datetime.timedelta(days=1)
# combine and make string
date_range_start_formatted = date_range_start.strftime("%m/%d/%Y")
date_range_end_formatted = date_range_end.strftime("%m/%d/%Y")
date_range = date_range_start_formatted + " - " + date_range_end_formatted 

In [ ]:
### PDF and CSV generation loop

# global
for i in filter_list:

### PDF portion
    pdf_params['filter'] = f'vf_xxx={parse.quote(i)}' # xxx = name of field used for filter
    pdf = conn.query_view_pdf(view_id=dashboard_view_id, parameter_dict=pdf_params)
    pdf_new = open(os.path.join(path,f'{i}.pdf'), 'wb')
    pdf_new.write(pdf.content)    

### CSV Portion
    csv_params['filter'] = f'vf_xxx={parse.quote(i)}' # xxx = name of field used for filter
    report_data = conn.query_view_data(view_id=report_view_id, parameter_dict=csv_params)
    report_df = pd.read_csv(io.StringIO(report_data.content.decode('utf-8')))
    # prep data
    report_data_clean = pd.merge(
        report_df.pivot(index = 'Row', columns = 'Measure Names', values = 'Measure Values'),
        report_df.drop(columns = ['Measure Names', 'Measure Values']).drop_duplicates(),
        on = 'Row').drop(columns = ['xxx', 'xxx']) # whatever fields you want to remove
    cols = ['xxx', 'xxx', 'xxx'] # reorder fields
    report_data_clean = report_data_clean[cols]

 # sort by date
    report_data_clean['Date'] = pd.to_datetime(report_data_clean['Date'], format='%m/%d/%Y')
    report_data_clean = report_data_clean.sort_values('Date')
    report_data_clean['Date'] = report_data_clean['Date'].dt.date
    
    
### FORMATTING

# Use xlsxwriter to format the excel output. Very customizable

    # replace commas from dollar and qty
    report_data_clean['Dollar Amt'] = report_data_clean['Dollar Amt'].astype(str)
    report_data_clean['Dollar Amt'] = report_data_clean['Dollar Amt'].apply(lambda x : x.replace(',',''))

    # convert Dollar Amt to float
    report_data_clean['Dollar Amt'] = report_data_clean['Dollar Amt'].astype(float)

    # Get access to worksheet object
    writer = pd.ExcelWriter(os.path.join(path,f'{i}.xlsx'), engine = 'xlsxwriter')
    report_data_clean.to_excel(writer, index = False, sheet_name = 'Sheet')
    workbook = writer.book
    worksheet = writer.sheets['Sheet']

    # set size
    worksheet.set_zoom(90)

    # header formatting
    header_format = workbook.add_format({
        "valign": "vcenter",
        "align": "center",
        "bg_color": "#951F06",
        "bold": True,
        'font_color': '#FFFFFF'
    })

    # add title
    title = "Invoice Listing for " + f"{i}"
    #merge cells
    format = workbook.add_format()
    format.set_font_size(20)
    format.set_font_color("#333333")

    subheader = f"{date_range}"
    worksheet.merge_range('A1:K1', title, format)
    worksheet.merge_range('A2:K2', subheader)
    worksheet.set_row(2,15)

    # put it all together
    # write the column headers with defined format

    for col_num, value in enumerate(report_data_clean.columns.values):
        worksheet.write(2, col_num, value, header_format)

    # number format for columns with money
    money_fmt = workbook.add_format({'num_format': '$#,##0.00'})
    # total formatting - Dollars
    total_fmt = workbook.add_format({'align': 'right', 'num_format': '$#,##0.00',
                                 'bold': True, 'bottom':6})
    # total formatting - Quantity
    total_fmt_q = workbook.add_format({'align': 'right', 'num_format': '#,##0',
                                 'bold': True, 'bottom':6})

    # adjust column width
    worksheet.set_column('A:K', 20)

    # money columns
    worksheet.set_column('J:K', 20, money_fmt)

    number_rows = len(report_data_clean.index)

    # add total money rows
    for column in range(9,11):
        # determine where to placce the formula
        cell_location = xl_rowcol_to_cell(number_rows+1, column)
        # Get the range to use for the sum formula
        start_range = xl_rowcol_to_cell(3,column)
        end_range = xl_rowcol_to_cell(number_rows, column)
        # Construct and write formula
        formula = "=SUM({:s}:{:s})".format(start_range, end_range)
        worksheet.write_formula(cell_location, formula, total_fmt)

    # add total qty row
    for column in range(8,9):
        # determine where to place the formula
        cell_location = xl_rowcol_to_cell(number_rows+1, column)
        # Get the range to use for the sum formula
        start_range = xl_rowcol_to_cell(3,column)
        end_range = xl_rowcol_to_cell(number_rows, column)
        # Construct and write formula
        formula = "=SUM({:s}:{:s})".format(start_range, end_range)
        worksheet.write_formula(cell_location, formula, total_fmt_q)

    # add a total label
    worksheet.write_string(number_rows+1, 7, "Total", total_fmt)

    # advance output
    writer.save()